In [ ]:
!python -m pip install --user -r ../requirements.txt

In [ ]:
!python -m pip install --user  opencv-python-headless

In [ ]:
!python -m pip install --user layoutparser # Install the base layoutparser library with  
!python -m pip install --user  "layoutparser[layoutmodels]" # Install DL layout model toolkit 
!python -m pip install --user "layoutparser[ocr]" # Install OCR toolkit

In [ ]:
!python -m pip install --user 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'

In [ ]:
import layoutparser as lp
import cv2
import numpy as np
import pdf2image
import os

In [ ]:
def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file,
        grayscale=False,thread_count=os.cpu_count())

In [ ]:
img = pdf_to_img("Liste_admissibilite_admn_ext_2023_public.pdf")

In [ ]:
model = lp.models.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
                                 label_map={0: "Text", 1: "Title", 2: "List", 
                                 3:"Table",4:"Figure"})

In [ ]:
mynumpyarray=np.asarray(img[0])
layout = model.detect(mynumpyarray)

In [ ]:
lp.draw_box(mynumpyarray, layout, box_width=3)


In [ ]:
title_blocks = lp.Layout([b for b in layout if b.type=='Title'])

In [ ]:
title_blocks

In [ ]:
ocr_agent = lp.TesseractAgent(languages='fra') 

In [ ]:
for block in title_blocks:
    segment_image = (block
                       .pad(left=5, right=5, top=5, bottom=5)
                       .crop_image(mynumpyarray))
        # add padding in each image segment can help
        # improve robustness 
        
    text = ocr_agent.detect(segment_image)
    block.set(text=text, inplace=True)

In [ ]:
for txt in title_blocks.get_texts():
    print(txt, end='\n---\n')